## Data frame of neighborhood information with postal code data

Download and import required libraries

In [42]:
import numpy as np
import requests
!conda install -c conda-forge lxml --yes
import lxml.html as lh
import pandas as pd

Solving environment: done

# All requested packages already installed.



Defining column names

In [43]:
column_names = ['PostalCode','Borough', 'Neighborhood']

Create a handle, page, to handle the contents of the website

In [44]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

Store the contents of the website under doc and Parse data that are stored between \<tr\>..\</tr\> of HTML

In [45]:
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Check the length of the first 12 rows

In [46]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Store header as first element

In [47]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    #print '%d:"%s"'%(i,name)
    col.append((name,[]))

Since out first row is the header, data is stored on the second row onwards

In [48]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [49]:
[len(C) for (title,C) in col]

[181, 181, 181]

Assingning dictionary Dict to a dataframe df

In [50]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [51]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [52]:
df.shape

(181, 3)

Remove \n tag from each cell

In [53]:
df['Postal Code\n']= df['Postal Code\n'].map(lambda x: x.rstrip('\n'))
df['Borough\n']= df['Borough\n'].map(lambda x: x.rstrip('\n'))
df['Neighborhood\n']= df['Neighborhood\n'].map(lambda x: x.rstrip('\n'))

In [54]:
df.columns = column_names

In [55]:
df.tail()

,PostalCode,Borough,Neighborhood
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned
180,,Canadian postal codes,


Removing unsuitable row index 180 and remove rows where Borough is Not assigned  

In [56]:
last_row_removed =df.drop([180])
refined_data = last_row_removed[last_row_removed.Borough != 'Not assigned']

In [57]:
refined_data.shape

(103, 3)

In [58]:
scrapedDataFrame = refined_data.reset_index().drop('index',axis=1)

In [59]:
scrapedDataFrame.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4G,North York,Downsview
freq,1,24,4


In [60]:
scrapedDataFrame.shape

(103, 3)

In [61]:
scrapedDataFrame

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Downloading Geospatial_data csv file

In [62]:
!wget -O geospatial_data.csv http://cocl.us/Geospatial_data

--2020-06-16 15:13:59--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 158.85.108.86, 158.85.108.83, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.86|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-06-16 15:13:59--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-16 15:14:03--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-06-16 15:14:03--  https://ibm.box.com/public

In [63]:
df_postalcodes = pd.read_csv('geospatial_data.csv')
df_postalcodes.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Update scrapedDataFrame with longitude and latitude of Postal codes

In [64]:
for x in range(0,scrapedDataFrame.shape[0],1):
    p_code=scrapedDataFrame.iloc[x]['PostalCode']
    scrapedDataFrame.loc[x, 'Latitude'] = df_postalcodes.loc[df_postalcodes['Postal Code'] == p_code]['Latitude'].item()
    scrapedDataFrame.loc[x, 'Longitude'] = df_postalcodes.loc[df_postalcodes['Postal Code'] == p_code]['Longitude'].item()

In [65]:
scrapedDataFrame

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [66]:
scrapedDataFrame.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [69]:
scrapedDataFrame.to_csv('output.csv',index=False)